In [1]:
import pandas as pd
import numpy as np

In [2]:
# Before you doing this, you need to remove the first line(header) of the below file
match_best = pd.read_csv('Algorithm/match_best.csv', header=None)
match_second = pd.read_csv('Algorithm/match_second.csv', header=None)
match_third = pd.read_csv('Algorithm/match_third.csv', header=None)
raw_data = pd.read_csv('Algorithm/raw_data.csv', header=None)
Training_data = pd.read_csv('Algorithm/training_data.csv', header=None)
Testing_data = pd.read_csv('Algorithm/testing_data.csv', header=None)

In [3]:
# Training_data # 17583 rows × 117 columns

In [4]:
U = match_second - match_third

In [5]:
# U # 1959 rows × 116 columns
numOfColumns = U.shape[1]

In [6]:
# INPORTANT : numOfColumns + 1 
data = np.identity( numOfColumns + 1 )   # with topic features

In [7]:
# data

In [8]:
I = pd.DataFrame(data=data[1:,1:])

In [9]:
# I

In [10]:
parameter_l = 0.5
parameter_u = 0.5
# 0.01,1.32,0.04
C = pd.concat([parameter_l * I, parameter_u * U],ignore_index=True)

In [11]:
# C

In [12]:
def S(x) :
    if x > -1 and x < 1:
        return x
    elif x >= 1 :
        return 1
    else :
        return -1

# test the S(x)
# print S(0)
# print S(0.5)
# print S(-0.7)
# print S(-2)
# print S(1.8)

In [13]:
random_num = np.random.uniform(-1, 1, size=numOfColumns)

In [14]:
w = pd.DataFrame(data=random_num)

In [15]:
# w # 116 rows × 1 columns

In [16]:
parameter_g = 0.04
#0.01,1.32,0.04
C_Dot_W = C.dot(w) / parameter_g

In [17]:
# C_Dot_W

In [18]:
A = C_Dot_W[0].apply(S)

In [19]:
# A

In [20]:
# C_T : C 's transpose
C_T = C.transpose()

In [21]:
C_T_Dot_A = 0.5 * C_T.dot(A)

In [22]:
# C_T_Dot_A

In [23]:
# Training_data # 17583 rows × 118 columns

In [24]:
# with topic features
X = Training_data.ix[0:, 0:numOfColumns-1]
Y = Training_data.ix[0:, numOfColumns]

In [25]:
# X

In [26]:
# Y 

In [27]:
X_Mul_Y = X.mul(Y, axis=0)

In [28]:
# X_Mul_Y

In [29]:
Sum_X_Mul_Y = X_Mul_Y.sum()

In [30]:
# Sum_X_Mul_Y

In [31]:
Target = Sum_X_Mul_Y - C_T_Dot_A

In [32]:
Target = Target.as_matrix()

In [33]:
# Target

In [34]:
X_T = X.transpose()

In [35]:
X_T_Dot_X = X_T.dot(X)

In [36]:
# X_T_Dot_X

In [37]:
Parameter = X_T_Dot_X.as_matrix()

In [38]:
# Parameter

In [39]:
w = np.linalg.solve(Parameter, Target)

In [40]:
# w

# Optimaization Algorithm

In [41]:
iterations = 0
delta_A = 1000
delta_w = 1000
bound = 0.0001

while delta_A > bound and delta_w > bound :
    iterations += 1
    
    C_Dot_W = C.dot(w) / parameter_g
    
    # update vector A
    A_now = C_Dot_W.apply(S)
    
    C_T_Dot_A = 0.5 * C_T.dot(A_now)
    Target = Sum_X_Mul_Y - C_T_Dot_A
    Target = Target.as_matrix()
    
    # update vector w
    w_now = np.linalg.solve(Parameter, Target)
    
    delta_A = np.linalg.norm(A_now-A)
    delta_w = np.linalg.norm(w_now-w)
    
    A = A_now
    w = w_now
    
    print 'iterations:' + str(iterations) + ' ' + \
            'delta_A:' + str(delta_A) + ' ' + \
            'delta_w:' + str(delta_w)  
print w

iterations:1 delta_A:64.8438601389 delta_w:0.265587550932
iterations:2 delta_A:21.2882972716 delta_w:0.0796696919013
iterations:3 delta_A:11.0636930671 delta_w:0.0383771907771
iterations:4 delta_A:6.21573367738 delta_w:0.0215558366858
iterations:5 delta_A:3.62317312506 delta_w:0.0123581220322
iterations:6 delta_A:2.12999047779 delta_w:0.00731661044055
iterations:7 delta_A:1.26125601192 delta_w:0.00433013062596
iterations:8 delta_A:0.748966561632 delta_w:0.00258354581944
iterations:9 delta_A:0.444593983392 delta_w:0.00153858926352
iterations:10 delta_A:0.26438943044 delta_w:0.000917637962364
iterations:11 delta_A:0.157322757278 delta_w:0.0005470594415
iterations:12 delta_A:0.0936354416734 delta_w:0.000326041789168
iterations:13 delta_A:0.055738363397 delta_w:0.000194268000328
iterations:14 delta_A:0.0331815275199 delta_w:0.000115727232419
iterations:15 delta_A:0.0197538941416 delta_w:6.89277326791e-05
[-0.06353807 -0.3092523   0.02876595 -0.13703534  0.3466891   0.01154732
 -0.05004805 

In [42]:
test_features = Testing_data.ix[0:, 0:numOfColumns-1]
test_flag = Testing_data.ix[0:, numOfColumns]

In [43]:
# Testing_data
# test_features

In [44]:
result = np.dot(test_features, w)

In [45]:
print result.max()
print result.min()
#print result
predict_flag = result > 0

1.88492610403
-2.12791665436


In [46]:
len(predict_flag)
print predict_flag[0]
if predict_flag[0] == 1:
    print "True == 1"

True
True == 1


In [47]:
#test_flag

true_positive = 0.0
true_negative = 0.0
false_positive = 0.0
false_negative = 0.0

for i in range( len(predict_flag) ):
    if predict_flag[i] == True and test_flag[i] == 1 :
        true_positive += 1
    if predict_flag[i] == False and test_flag[i] == -1 :
        true_negative += 1
    if predict_flag[i] == True and test_flag[i] == -1 :
        false_positive += 1
    if predict_flag[i] == False and test_flag[i] == 1 :
        false_negative += 1
        
#print true_positive
#print true_negative
#print false_positive
#print false_negative

ACC = (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)
PRE = true_positive / (true_positive + false_positive)
REC = true_positive / (true_positive + false_negative)
F1 = 2*( PRE * REC) / (PRE + REC)
print ACC
print PRE
print REC
print F1

0.994020926756
0.9921875
0.996078431373
0.994129158513


In [51]:
def Store_Model( Weights , File ) :
    # Write Model's performance to the File
    File.write(str(F1*100) + '%')
    
    # Write coefficients to the Filse 
    coefficients = Weights
    for i in range( len(coefficients) ) :
        # print i,coefficients[i]
        File.write(',' + str(coefficients[i]) )

In [52]:
with open('./MODEL/MODEL_01.txt', 'w') as model01 :
    Store_Model(w, model01)